In [ ]:
using namespace System.Linq.Expressions
# this records [1] mandatory depedencies [2] Which versions were used
# [3] pwsh version, and [4] date executed 
Get-Date | Join-String -op 'Last Executed: ' { $_.ToString('U') }

$PSVersionTable | Join-String PSVersion -op 'Pwsh: '

Import-Module -ea "stop" -passThru 'ImpliedReflection', 'ClassExplorer'
| Join-String -op 'Modules: ' { $_.Name, $_.Version -join ' = ' } -sep ', '

Enable-ImpliedReflection -YesIKnowIShouldNotDoThis

Last Executed: Wednesday, August 3, 2022 4:04:27 PM
Pwsh: 7.2.5
Modules: ImpliedReflection = 1.0.0, ClassExplorer = 2.3.3


In [ ]:

$binderType = Find-Type -IncludeNonPublic PSBinaryOperationBinder
$binder = $binderType::Get(
    [ExpressionType]::GreaterThan,
    <# ignoreCase: #> $true,
    <# scalarCompare: #> $false)

$binderResult = $binder.Bind(
    @('a', 'b'),
    ([Expression]::Parameter([string], 'lhs'), [Expression]::Parameter([string], 'rhs')),
    [Expression]::Label([object]))

$binderResult | Format-ExpressionTree

if ((lhs is string) && (rhs is string))
{
    return StringOps.Compare(
        lhs,
        rhs,
        Fake.Const<CultureInfo>("_empty<CultureInfo>"),
        CompareOptions.IgnoreCase) > 0;
}


In [ ]:
$binderType | ft -auto


   Namespace: System.Management.Automation.[
37mLanguage

Access            Modifiers             Name
------            ---------             ----
internal sealed class 



In [ ]:
$binder


ReturnType       : System.Object
Operation        : GreaterThan
IsStandardBinder : True
_ignoreCase      : True
_scalarCompare   : False
_version         : 0
_compareDelegate : 
Cache            : {[System.Func`4[System.Runtime.CompilerServices.CallSite,System.Object,System.In
                   t32,System.Object], System.Runtime.CompilerServices.RuleCache`1[System.Func`4[Sy
                   stem.Runtime.CompilerServices.CallSite,System.Object,System.Int32,System.Object]
                   ]]}




In [ ]:
$binderResult


NodeType                : Conditional
Type                    : System.Void
Test                    : ((lhs TypeEqual String) AndAlso (rhs TypeEqual String))
IfTrue                  : return UnnamedLabel_0 (Convert((Compare(lhs, rhs, , IgnoreCase) > 0), 
                          Object))
IfFalse                 : default(Void)
CanReduce               : False
<Test>k__BackingField   : ((lhs TypeEqual String) AndAlso (rhs TypeEqual String))
<IfTrue>k__BackingField : return UnnamedLabel_0 (Convert((Compare(lhs, rhs, , IgnoreCase) > 0), 
                          Object))




#### Answer

```md
example of how you can get the exact call from a binder
```

[Source: Discord thread](https://discord.com/channels/180528040881815552/447476117629304853/1004411457691193405), and [earlier segment](https://discord.com/channels/180528040881815552/447476117629304853/1004401433484988446)

I started asking if you could find the entry point for what function resolves comparing strings in Pwsh. 
#### I tried: 
```ps1
Pwsh> ScriptBlockDisassembler\Get-ScriptBlockDisassembly -ScriptBlock { 'a' -gt 'b' } -Minimal
```
```csharp
try
{
    Fake.Dynamic<Action<CallSite, object, Pipe, ExecutionContext>>(PSPipeWriterBinder.Get())(
        Fake.Dynamic<Func<CallSite, string, string, object>>(PSBinaryOperationBinder.Get(ExpressionType.GreaterThan, ignoreCase: true,scalarCompare: false))("a", "b"),
        funcContext._outputPipe,
        context);
}
```


In [ ]:
# generate metadata 

gcm 'Enable-ImpliedReflection', 'Find-Type'
| Join-String { $_.Source, $_.Name -join '\' } -sep ', '

gcm 'Enable-ImpliedReflection', 'Find-Type'
| Join-String Source -sep ', ' -single -op 'Import-Module -ea "stop" -passThru '

ImpliedReflection\Enable-ImpliedReflection, ClassExplorer\Find-Type
Import-Module -ea "stop" -passThru 'ImpliedReflection', 'ClassExplorer'
